# Preprocessing and Data cleaning

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
import numpy as np

## Read in raw data and Search the sentiment dict

In [ ]:
file = 'raw_data/TG.csv'
df = pd.read_csv(file, encoding='utf-8')
dict_df = pd.read_csv('inquirerbasic.csv')

In [ ]:
positive_words = dict_df[dict_df['Positiv'].notnull()]['Entry'].str.lower().tolist()
negative_words = dict_df[dict_df['Negativ'].notnull()]['Entry'].str.lower().tolist()
strong_words = dict_df[dict_df['Strong'].notnull()]['Entry'].str.lower().tolist()
weak_words = dict_df[dict_df['Weak'].notnull()]['Entry'].str.lower().tolist()

In [ ]:
def count_positive_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the positive words
    return sum(token in positive_words for token in tokens)


In [ ]:
def count_negative_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the negative words
    return sum(token in negative_words for token in tokens)

In [ ]:
def count_strong_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the strong words
    return sum(token in strong_words for token in tokens)


In [ ]:
def count_weak_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the weak words
    return sum(token in weak_words for token in tokens)

In [ ]:
# Add a new column 'positive_word_count' to the DataFrame
df['positive_word_count'] = df['body'].apply(count_positive_words)
# Add a new column 'negative_word_count' to the DataFrame
df['negative_word_count'] = df['body'].apply(count_negative_words)
df['strong_word_count'] = df['body'].apply(count_strong_words)
df['weak_word_count'] = df['body'].apply(count_weak_words)

In [ ]:
# df['positive_score'] = df['positive_word_count'] / df['length']
# df['negative_score'] = df['negative_word_count'] / df['length']
# df['strong_score'] = df['strong_word_count'] / df['length']
# df['weak_score'] = df['weak_word_count'] / df['length']

In [ ]:
# columns_to_drop = ['positive_word_count', 'negative_word_count', 'strong_word_count', 'weak_word_count']
# df = df.drop(columns=columns_to_drop)

df.to_csv('TG_processed.csv', index=False)

## Data Cleaning 

### News

In [ ]:
nyt = pd.read_csv('processed_data/NYT_processed.csv')
columns_to_drop = ['section', 'body', 'subject', 'title']

# drop NA values
nyt.dropna(inplace=True)

# drop the unnecessary columns
nyt.drop(columns=columns_to_drop, inplace=True)

# convert the 'date' column to datetime format
nyt['date'] = pd.to_datetime(nyt['date'])

# sort the dataframe by date
nyt.sort_values('date', inplace=True)

nyt

# check if there are any duplicated items
# any_duplicates = nyt['date'].duplicated().any()

# print(any_duplicates)

In [ ]:
tg = pd.read_csv('processed_data/TG_processed.csv')
columns_to_drop = ['section', 'body', 'subject', 'title']

# drop NA values
tg.dropna(inplace=True)

# drop the unnecessary columns
tg.drop(columns=columns_to_drop, inplace=True)

# convert the 'date' column to datetime format
tg['date'] = pd.to_datetime(tg['date'])

# sort the dataframe by date
tg.sort_values('date', inplace=True)

tg


In [ ]:
times = pd.read_csv('processed_data/TIMES_processed.csv', encoding='latin-1')
columns_to_drop = ['section', 'body', 'subject', 'title']

# drop NA values
times.dropna(inplace=True)

# drop the unnecessary columns
times.drop(columns=columns_to_drop, inplace=True)

# convert the 'date' column to datetime format
times['date'] = pd.to_datetime(times['date'])

# sort the dataframe by date
times.sort_values('date', inplace=True)

times


In [ ]:
cd = pd.read_csv('processed_data/CD_processed.csv')
columns_to_drop = ['body', 'subject', 'title']
cd.dropna(inplace=True)
cd.drop(columns=columns_to_drop, inplace=True)
cd['date'] = pd.to_datetime(cd['date'])
cd.sort_values('date', inplace=True)
cd

### Stock Exchange Index

In [ ]:
nyse = pd.read_csv('raw_data/NYSE_index.csv')
nyse['Date'] = pd.to_datetime(nyse['Date'])
nyse.info()
nyse

In [ ]:
nyse['Daily_return'] = nyse['Close'].pct_change()
nyse['Trend'] = np.where(nyse['Daily_return'] > 0, 'UP', 'DOWN')
nyse

In [ ]:
def trend_duration(trend_column):
    trend_duration = [1]  # Start with 1 for the first day
    for i in range(1, len(trend_column)):
        if trend_column[i] == trend_column[i-1]:  # If the trend is the same as the previous day's
            trend_duration.append(trend_duration[i-1] + 1)  # Add 1 to the previous day's count
        else:  # If the trend changed
            trend_duration.append(1)  # Start a new count
    return trend_duration

# Apply the function to the 'Trend' column
nyse['Trend_duration'] = trend_duration(nyse['Trend'].tolist())
nyse

In [ ]:
# processed NYSE index data
nyse.to_csv('nyse_processed.csv')

In [ ]:
sse = pd.read_csv('raw_data/SSE_index.csv')
sse['Date'] = pd.to_datetime(sse['Date'])
sse.info()

In [ ]:
sse['Daily_return'] = sse['Close'].pct_change()
sse['Trend'] = np.where(sse['Daily_return'] > 0, 'UP', 'DOWN')
sse

# Simple Data Analysis

## DA on news reports

### Descriptive Info

In [ ]:
import seaborn as sns
import pandas as pd
result = pd.read_csv('cleaned_INFO.csv')
result.describe()

In [ ]:
# average lengths of different publications
avg_length = result.groupby('source')['length'].mean()
avg_length

In [ ]:
# variation
var_length = result.groupby('source')['length'].std()
var_length

In [ ]:
# Compute the maximum report length for each source
max_length = result.groupby('source')['length'].max()

max_length

In [ ]:
# Compute the minimum report length for each source
min_length = result.groupby('source')['length'].min()

min_length

In [ ]:
# from scipy.stats import zscore

# # Compute the z-scores for the 'length' column
# result['length_zscore'] = result.groupby('source')['length'].transform(zscore)

# # result

In [ ]:
# Count the number of reports from each source for each date
counts = result.groupby(['source', 'date']).size().reset_index(name='count')

# Filter the DataFrame to only show the rows where 'report_count' is more than 1
duplicated = counts[counts['count'] > 1]

# Print the result
duplicated

In [ ]:
sum_up = result.groupby(['date', 'source']).sum()
sum_up = sum_up.reset_index()
sum_up.to_csv('processed_INFO.csv')
sum_up

### Content Length Analysis

In [ ]:
sum_up['length'].describe()

In [ ]:
sum_up.groupby('source')['length'].mean()

In [ ]:
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
# Histogram
plt.figure(figsize=(10,6))
sns.histplot(sum_up['length'], bins=30, kde=True)
plt.title('Distribution of Report Lengths')
plt.xlabel('Length')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='source', y='length', data=sum_up)
plt.title('Report Lengths by Source')
plt.xlabel('Source')
plt.ylabel('Length')
plt.show()

### Time Series Analysis

In [ ]:
# Trend of report length over time
plt.figure(figsize=(10,6))
sns.lineplot(x='date', y='length', data=sum_up)
plt.title('Trend of Report Length Over Time')
plt.xlabel('Date')
plt.ylabel('Average Length')
plt.show()

# Trend of report length over time for each source
df_grouped = sum_up.groupby(['date', 'source'])['length'].mean().reset_index()
plt.figure(figsize=(10,6))
sns.lineplot(x='date', y='length', hue='source', data=df_grouped)
plt.title('Trend of Report Length Over Time for Each Source')
plt.xlabel('Date')
plt.ylabel('Average Length')
plt.show()


In [ ]:
# Average sentiment by source
source_sentiments = sum_up.groupby('source')[['positive_word_count', 'negative_word_count']].mean()
print(source_sentiments)

# Plot
source_sentiments.plot(kind='bar', stacked=True)
plt.title('Average Sentiment by Source')
plt.xlabel('Source')
plt.ylabel('Average Word Count')
plt.show()


In [ ]:
# Correlation analysis
correlation_matrix = sum_up.corr()
correlation_matrix


In [ ]:
# Correlation heatmap
plt.figure(figsize=(10,6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Plot positive_word_count over time
sum_up.groupby('date')['positive_word_count'].mean().plot()
plt.title('Average Positive Word Count Over Time')
plt.ylabel('Average Positive Word Count')
plt.show()

# Plot negative_word_count over time
sum_up.groupby('date')['negative_word_count'].mean().plot()
plt.title('Average Negative Word Count Over Time')
plt.ylabel('Average Negative Word Count')
plt.show()


In [ ]:
from pandas.plotting import autocorrelation_plot

# Autocorrelation plot for the report length
autocorrelation_plot(sum_up['length'])
plt.title('Autocorrelation Plot for Report Length')
plt.show()


### Sentiment Analysis

In [ ]:
sum_up['positive_score'] = sum_up['positive_word_count']/sum_up['length']
sum_up['negative_score'] = sum_up['negative_word_count']/sum_up['length']
sum_up.head()

In [ ]:
average_scores_by_source = sum_up.groupby('source')[['positive_score', 'negative_score']].mean()
average_scores_by_source

## DA on stock exchange

In [ ]:
ny = pd.read_csv('nyse_processed.csv', index_col=0)
ny['Date'] = pd.to_datetime(ny['Date'])

In [ ]:
# about daily return
std_dev = ny['Daily_return'].std()
mean_return = ny['Daily_return'].mean()
skewness = ny['Daily_return'].skew()
kurtosis = ny['Daily_return'].kurtosis()
max_return = ny['Daily_return'].max()
min_return = ny['Daily_return'].min()

print('max:', max_return)
print('min:', min_return)
print('kurtosis:', kurtosis)
print('skewness:', skewness)
print('mean_return:', mean_return)
print('std_dev:', std_dev)

In [ ]:
# 1-5 day correlation
for i in range(1, 6):
    lagged_returns = ny['Daily_return'].shift(i)
    correlation = ny['Daily_return'].corr(lagged_returns.dropna())  # Remove missing values before calculating correlation
    print(f'Correlation of Daily Return with {i}-day Lag: {correlation}')


### Time Series Plot

In [ ]:
# CLose
ny.set_index('Date', inplace=True)
ny['Close'].plot(figsize=(14, 7))

plt.title('Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# Daily return
ny['Daily_return'].plot(figsize=(14, 7))

plt.title('Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
ny

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlation matrix
correlation_matrix = ny.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
from pandas.plotting import autocorrelation_plot

# Autocorrelation plot for the report length
autocorrelation_plot(ny['Close'])
plt.title('Autocorrelation Plot for Close price')
plt.show()

## Grouped Analysis(join news dataset and stock dataset)

In [ ]:
sum_up.head()

In [ ]:
ny.head()

In [ ]:
news_without_cd = sum_up[sum_up['source'] != 'China Daily']
news_without_cd.head()

In [ ]:
df = news_without_cd.drop(columns=['source','positive_score','negative_score'])
df

In [ ]:
duplicated_dates = df[df['date'].duplicated(keep=False)]
duplicated_dates

In [ ]:
df_grouped = df.groupby('date').agg({
    'length': 'sum',
    'positive_word_count': 'sum',
    'negative_word_count': 'sum',
    'strong_word_count':'sum',
    'weak_word_count':'sum'
}).reset_index()
df_grouped

In [ ]:
# df_grouped.drop(columns=['strong_word_count','weak_word_count'], inplace=True)
df_grouped['date'] = pd.to_datetime(df_grouped['date'])
df_grouped.info()

In [ ]:
pd.read_csv('raw_data/NYSE_index.csv')

In [ ]:
ny = pd.read_csv('nyse_processed.csv', index_col=0)
ny['Date'] = pd.to_datetime(ny['Date'])
ny = ny.rename(columns={'Date':'date'})
ny.info()

In [ ]:
df_merged = df_grouped.merge(ny, on='date', how='inner')

In [ ]:
df_merged.head()

In [ ]:
df_merged['negative_score'] = df_merged['negative_word_count']/df_merged['length']
df_merged['positive_score'] = df_merged['positive_word_count']/df_merged['length']
df_merged.head()

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,10))
cor = df_merged.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()


# NEW NEW NEW NEW NEW

In [81]:
import pandas as pd
wall = pd.read_csv('raw_data/INFO_corpus/wall_street_journal.csv', encoding='latin-1')
tgnyt = pd.read_csv('raw_data/INFO_corpus/tgandnyt.csv', encoding='latin-1')
dict_df = pd.read_csv('inquirerbasic.csv')
wall.head()

C:\Users\20315\AppData\Local\Temp\ipykernel_17168\3819209801.py:4: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_df = pd.read_csv('inquirerbasic.csv')


,body,date,length
0,WASHINGTON -- Senate negotiators and the Trump...,25-Mar-20,639
1,Athletes around the world breathed a sigh of r...,25-Mar-20,562
2,"This month, tens of thousands marched in Spain...",25-Mar-20,790
3,WASHINGTON -- The Environmental Protection Age...,25-Mar-20,417
4,LOS ANGELES -- Southern California's always-ja...,25-Mar-20,995


In [82]:
tgnyt.head()

,body,date,length
0,A number of other changes were announced inclu...,5-May-20,633.0
1,A small Buffalo manufacturer has had to trim i...,5-May-20,1607.0
2,An internal Trump administration model project...,5-May-20,1674.0
3,As businesses contemplate the return of worker...,5-May-20,1390.0
4,Largely confined to their homes and worried ab...,5-May-20,1507.0


In [84]:
info = pd.concat([wall, tgnyt])

In [86]:
info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13053 entries, 0 to 6172
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    12994 non-null  object
 1   date    12875 non-null  object
 2   length  12912 non-null  object
dtypes: object(3)
memory usage: 407.9+ KB


In [87]:
info = info.dropna()
info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12849 entries, 0 to 6172
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    12849 non-null  object
 1   date    12849 non-null  object
 2   length  12849 non-null  object
dtypes: object(3)
memory usage: 401.5+ KB


In [88]:
info.head()

,body,date,length
0,WASHINGTON -- Senate negotiators and the Trump...,25-Mar-20,639
1,Athletes around the world breathed a sigh of r...,25-Mar-20,562
2,"This month, tens of thousands marched in Spain...",25-Mar-20,790
3,WASHINGTON -- The Environmental Protection Age...,25-Mar-20,417
4,LOS ANGELES -- Southern California's always-ja...,25-Mar-20,995


In [89]:
positive_words = dict_df[dict_df['Positiv'].notnull()]['Entry'].str.lower().tolist()
negative_words = dict_df[dict_df['Negativ'].notnull()]['Entry'].str.lower().tolist()

In [90]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
import numpy as np

def count_positive_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the positive words
    return sum(token in positive_words for token in tokens)


def count_negative_words(body):
    # Check if body is not a string (it could be nan or a number)
    if not isinstance(body, str):
        return 0
    
    # Remove punctuation and convert the body to lowercase
    body = body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the body
    tokens = word_tokenize(body)
    
    # Count the negative words
    return sum(token in negative_words for token in tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20315\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [91]:
info['positive_word_count'] = info['body'].apply(count_positive_words)
# Add a new column 'negative_word_count' to the DataFrame
info['negative_word_count'] = info['body'].apply(count_negative_words)

In [92]:
info.head()

,body,date,length,positive_word_count,negative_word_count
0,WASHINGTON -- Senate negotiators and the Trump...,25-Mar-20,639,16,4
1,Athletes around the world breathed a sigh of r...,25-Mar-20,562,6,5
2,"This month, tens of thousands marched in Spain...",25-Mar-20,790,12,14
3,WASHINGTON -- The Environmental Protection Age...,25-Mar-20,417,12,6
4,LOS ANGELES -- Southern California's always-ja...,25-Mar-20,995,13,3


In [93]:
info['date'].describe()

count      12849
unique       396
top        2020"
freq        1703
Name: date, dtype: object

In [94]:
info1 = info
from dateutil.parser import parse

def is_date(string):
    try: 
        parse(string)
        return True
    except ValueError:
        return False

# Convert all dates to strings before trying to parse them
info1['date'] = info1['date'].astype(str)

# Create a boolean mask for the rows with good dates
mask = info1['date'].apply(is_date)

# Keep only the rows with good dates (and drop the rows with bad dates)
info1 = info1[mask]
info1




,body,date,length,positive_word_count,negative_word_count
0,WASHINGTON -- Senate negotiators and the Trump...,25-Mar-20,639,16,4
1,Athletes around the world breathed a sigh of r...,25-Mar-20,562,6,5
2,"This month, tens of thousands marched in Spain...",25-Mar-20,790,12,14
3,WASHINGTON -- The Environmental Protection Age...,25-Mar-20,417,12,6
4,LOS ANGELES -- Southern California's always-ja...,25-Mar-20,995,13,3
...,...,...,...,...,...
6161,When Chinese scientists alerted colleagues to ...,12-Dec-20,1772.0,34,22
6163,Theyâre loyal diligent â?and have unbeatabl...,12-Dec-20,1971.0,34,21
6166,Air quality found to be the same or worse than...,11-Dec-20,601.0,9,14
6171,Field hospitals are set up in California Texas...,11-Dec-20,944.0,21,15


In [95]:
# Create a function to handle multiple date formats
def parse_dates(date):
    for fmt in ('%b%d,%Y', '%d-%b-%y'):
        try:
            return pd.datetime.strptime(date, fmt)
        except ValueError:
            continue
    return np.nan

# Apply the function to the date column
info1['date'] = info1['date'].apply(parse_dates)


info1.head()

C:\Users\20315\AppData\Local\Temp\ipykernel_17168\3360093039.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return pd.datetime.strptime(date, fmt)
C:\Users\20315\AppData\Local\Temp\ipykernel_17168\3360093039.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info1['date'] = info1['date'].apply(parse_dates)


,body,date,length,positive_word_count,negative_word_count
0,WASHINGTON -- Senate negotiators and the Trump...,2020-03-25,639,16,4
1,Athletes around the world breathed a sigh of r...,2020-03-25,562,6,5
2,"This month, tens of thousands marched in Spain...",2020-03-25,790,12,14
3,WASHINGTON -- The Environmental Protection Age...,2020-03-25,417,12,6
4,LOS ANGELES -- Southern California's always-ja...,2020-03-25,995,13,3


In [96]:
info1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11092 entries, 0 to 6172
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   body                 11092 non-null  object        
 1   date                 11083 non-null  datetime64[ns]
 2   length               11092 non-null  object        
 3   positive_word_count  11092 non-null  int64         
 4   negative_word_count  11092 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 519.9+ KB


In [97]:
# Show rows where 'date' or 'length' is null
missing_dates_or_lengths = info1[info1['date'].isnull() | info1['length'].isnull()]
print(missing_dates_or_lengths)


                                                   body date length  \
1249  WASHINGTON -- U.S. production of crude oil wil...  NaT    369   
2886  Four years ago, Aaron Levie moved Box Inc., th...  NaT    921   
3204  With the sports world reeling from the coronav...  NaT   1063   
3759  Coronavirus cases surged in the U.S., outstrip...  NaT   1056   
5578  MITSUBISHI CHEMICAL\nForeign Executive\nAppoin...  NaT    805   
6017  The U.S. Centers for Disease Control and Preve...  NaT    604   
6769  NEW DELHI -- The companies behind two of the m...  NaT    389   
2935  Residents locked up in allegedly âinhumanâ?...  NaT  900.0   
4925  Attendance rates at their lowest in highCovid ...  NaT  537.0   

      positive_word_count  negative_word_count  
1249                    5                    6  
2886                   22                    3  
3204                   28                   15  
3759                   12                   11  
5578                    4                   

In [98]:
info1 = info1.dropna(subset=['date', 'length'])
info1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11083 entries, 0 to 6172
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   body                 11083 non-null  object        
 1   date                 11083 non-null  datetime64[ns]
 2   length               11083 non-null  object        
 3   positive_word_count  11083 non-null  int64         
 4   negative_word_count  11083 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 519.5+ KB


In [99]:
# Sort the DataFrame by date in ascending order
info1 = info1.sort_values(by='date')
info1 = info1.reset_index(drop=True)
# Check the result
info1.head()


,body,date,length,positive_word_count,negative_word_count
0,The deadly coronavirus outbreak is sparking bi...,2020-01-23,386,5,10
1,BEIJING -- It didn't take long to identify the...,2020-01-27,1599,23,32
2,"BEIJING -- The mayor of Wuhan, the city at the...",2020-01-28,1094,30,25
3,Investors who began the year feeling largely s...,2020-01-29,675,7,18
4,Commodities including crude oil and copper fel...,2020-01-31,530,0,14


In [100]:
columns_to_drop = ['body']
info1.drop(columns=columns_to_drop, inplace=True)
info1.head()

,date,length,positive_word_count,negative_word_count
0,2020-01-23,386,5,10
1,2020-01-27,1599,23,32
2,2020-01-28,1094,30,25
3,2020-01-29,675,7,18
4,2020-01-31,530,0,14


In [101]:
info1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11083 entries, 0 to 11082
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 11083 non-null  datetime64[ns]
 1   length               11083 non-null  object        
 2   positive_word_count  11083 non-null  int64         
 3   negative_word_count  11083 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 346.5+ KB


In [102]:
info1['length'] = info1['length'].astype(int)
info1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11083 entries, 0 to 11082
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 11083 non-null  datetime64[ns]
 1   length               11083 non-null  int32         
 2   positive_word_count  11083 non-null  int64         
 3   negative_word_count  11083 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 303.2 KB


In [103]:
# 这一步有问题
clean_grouped = info1.groupby('date').agg({
    'length': 'sum',

}).reset_index()
clean_grouped.head()
clean_grouped.to_csv('cleaned_info.csv', index=False)

In [104]:
clean_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 333 non-null    datetime64[ns]
 1   length               333 non-null    int32         
 2   positive_word_count  333 non-null    int64         
 3   negative_word_count  333 non-null    int64         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 9.2 KB


In [105]:
clean_grouped['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 333 entries, 0 to 332
Series name: date
Non-Null Count  Dtype         
--------------  -----         
333 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.7 KB


In [29]:
ny = pd.read_csv('nyse_processed.csv', index_col=0)
ny['Date'] = pd.to_datetime(ny['Date'])
ny = ny.rename(columns={'Date':'date'})
ny.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 0 to 776
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            777 non-null    datetime64[ns]
 1   Open            777 non-null    float64       
 2   High            777 non-null    float64       
 3   Low             777 non-null    float64       
 4   Close           777 non-null    float64       
 5   Adj Close       777 non-null    float64       
 6   Volume          777 non-null    int64         
 7   Daily_return    776 non-null    float64       
 8   Trend           777 non-null    object        
 9   Trend_duration  777 non-null    int64         
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 66.8+ KB


In [106]:
clean_merged = clean_grouped.merge(ny, on='date', how='inner')
clean_merged.head(50)

,date,length,positive_word_count,negative_word_count,Open,High,Low,Close,Adj Close,Volume,Daily_return,Trend,Trend_duration
0,2020-01-23,386,5,10,14064.280273,14109.589844,14003.280273,14102.040039,14102.040039,3766710000,-0.000581,DOWN,1
1,2020-01-27,1599,23,32,13746.629883,13826.429688,13742.009766,13769.599609,13769.599609,3831050000,-0.014942,DOWN,3
2,2020-01-28,1094,30,25,13812.650391,13913.589844,13798.339844,13877.610352,13877.610352,3531570000,0.007844,UP,1
3,2020-01-29,675,7,18,13912.790039,13922.440430,13843.790039,13843.809570,13843.809570,3600250000,-0.002436,DOWN,1
4,2020-01-31,530,0,14,13783.809570,13788.219727,13573.040039,13614.099609,13614.099609,4529700000,-0.017878,DOWN,1
5,2020-02-05,814,10,12,13964.000000,14029.910156,13947.589844,14024.860352,14024.860352,4121480000,0.011687,UP,3
6,2020-02-06,2160,10,37,14066.900391,14069.330078,14023.480469,14034.950195,14034.950195,3887250000,0.000719,UP,4
7,2020-02-07,259,0,3,13987.509766,13987.509766,13912.040039,13931.929688,13931.929688,3733920000,-0.007340,DOWN,1
8,2020-02-12,2963,45,39,14117.830078,14148.240234,14108.519531,14136.980469,14136.980469,3930910000,0.005899,UP,3
9,2020-02-13,4127,48,59,14069.250000,14132.879883,14049.129883,14099.040039,14099.040039,3500890000,-0.002684,DOWN,1


In [107]:
clean_merged['negative_score'] = clean_merged['negative_word_count']/clean_merged['length']
clean_merged

,date,length,positive_word_count,negative_word_count,Open,High,Low,Close,Adj Close,Volume,Daily_return,Trend,Trend_duration,negative_score
0,2020-01-23,386,5,10,14064.280273,14109.589844,14003.280273,14102.040039,14102.040039,3766710000,-0.000581,DOWN,1,0.025907
1,2020-01-27,1599,23,32,13746.629883,13826.429688,13742.009766,13769.599609,13769.599609,3831050000,-0.014942,DOWN,3,0.020013
2,2020-01-28,1094,30,25,13812.650391,13913.589844,13798.339844,13877.610352,13877.610352,3531570000,0.007844,UP,1,0.022852
3,2020-01-29,675,7,18,13912.790039,13922.440430,13843.790039,13843.809570,13843.809570,3600250000,-0.002436,DOWN,1,0.026667
4,2020-01-31,530,0,14,13783.809570,13788.219727,13573.040039,13614.099609,13614.099609,4529700000,-0.017878,DOWN,1,0.026415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2020-12-24,13809,240,160,14398.620117,14398.620117,14324.769531,14382.500000,14382.500000,1883780000,-0.001120,DOWN,1,0.011587
229,2020-12-28,21767,351,309,14476.349609,14481.480469,14396.230469,14405.769531,14405.769531,3535460000,0.001618,UP,1,0.014196
230,2020-12-29,30067,522,359,14500.030273,14515.809570,14371.870117,14397.919922,14397.919922,3393290000,-0.000545,DOWN,1,0.011940
231,2020-12-30,30403,542,298,14447.959961,14520.570313,14447.959961,14477.480469,14477.480469,3154850000,0.005526,UP,1,0.009802


## test how many days lag have the most significance

In [32]:
# 1-5 day correlation
test = clean_merged
for i in range(1, 6):
    lagged_returns = test['Daily_return'].shift(i)
    correlation = test['Daily_return'].corr(lagged_returns.dropna())  # Remove missing values before calculating correlation
    print(f'Correlation of Daily Return with {i}-day Lag: {correlation}')

Correlation of Daily Return with 1-day Lag: -0.2707865580513764
Correlation of Daily Return with 2-day Lag: 0.2849232362226285
Correlation of Daily Return with 3-day Lag: -0.02352066316820056
Correlation of Daily Return with 4-day Lag: -0.12434942366017733
Correlation of Daily Return with 5-day Lag: 0.1911720919820946


In [34]:
import statsmodels.formula.api as smf
test['return_lag1'] = test['Daily_return'].shift(1)
test['return_lag2'] = test['Daily_return'].shift(2)
test['return_lag3'] = test['Daily_return'].shift(3)
test['return_lag4'] = test['Daily_return'].shift(4)
test['return_lag5'] = test['Daily_return'].shift(5)
# Models with 1 to 5 lags
for i in range(1, 6):
    # Create the formula for the model
    formula = 'Daily_return ~ negative_score + ' + ' + '.join(f'return_lag{j}' for j in range(1, i+1))

    # Fit the model
    model = smf.ols(formula=formula, data=test).fit()

    # Print the AIC and BIC
    print(f'Model with {i} lags: AIC = {model.aic}, BIC = {model.bic}')


Model with 1 lags: AIC = -1105.5077887484867, BIC = -1095.193550821717
Model with 2 lags: AIC = -1108.1735732550276, BIC = -1094.4386852408106
Model with 3 lags: AIC = -1103.0973082092687, BIC = -1085.9505800644965
Model with 4 lags: AIC = -1105.5204466522086, BIC = -1084.9707465473202
Model with 5 lags: AIC = -1100.4613784419187, BIC = -1076.5176334470127


Correlation of Daily Return with 1-day Lag: -0.2707865580513764
Correlation of Daily Return with 2-day Lag: 0.2849232362226285
Correlation of Daily Return with 3-day Lag: -0.02352066316820056
Correlation of Daily Return with 4-day Lag: -0.12434942366017733
Correlation of Daily Return with 5-day Lag: 0.1911720919820946
Model with 1 lags: AIC = -1105.5077887484867, BIC = -1095.193550821717
Model with 2 lags: AIC = -1108.1735732550276, BIC = -1094.4386852408106
Model with 3 lags: AIC = -1103.0973082092687, BIC = -1085.9505800644965
Model with 4 lags: AIC = -1105.5204466522086, BIC = -1084.9707465473202
Model with 5 lags: AIC = -1100.4613784419187, BIC = -1076.5176334470127

## We can find that 2 day lag has the most significance

# model of 1 day lag and 2 day lag

In [108]:
##### import pandas as pd
import numpy as np
import statsmodels.api as sm

df = clean_merged

# Assuming df is your DataFrame and 'Daily_return' and 'negative_word_count' are the columns with returns and sentiment scores respectively
df['return_lag1'] = df['Daily_return'].shift(1)
df['return_lag2'] = df['Daily_return'].shift(2)

# Drop the missing values that were created because of the lag
df = df.dropna()

# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     14.81
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           8.98e-07
Time:                        19:44:47   Log-Likelihood:                 560.52
No. Observations:                 231   AIC:                            -1115.
Df Residuals:                     228   BIC:                            -1105.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0004      0.001      0.265      

# add sentiments

In [109]:
# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2', 'negative_score']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     11.35
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           5.75e-07
Time:                        19:44:55   Log-Likelihood:                 562.56
No. Observations:                 231   AIC:                            -1117.
Df Residuals:                     227   BIC:                            -1103.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0146      0.007      2.

# Imaginative Corpus

In [62]:
img = pd.read_csv('raw_data/IMG_corpus/img_corpus.csv', encoding='latin-1')
img.head()

,body,date,length
0,I note that Keir Starmer has opened his accoun...,6-Apr-20,270
1,As Covid19 makes social distancing a matter of...,6-Apr-20,963
2,Britons will need to stay at home for weeks to...,6-Apr-20,646
3,Labourâs new leader exudes competence and id...,6-Apr-20,613
4,WHEN are news and magazinestyle programmes goi...,6-Apr-20,102


In [63]:
img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3551 entries, 0 to 3550
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    3243 non-null   object
 1   date    3535 non-null   object
 2   length  3551 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 83.4+ KB


In [64]:
# Show rows where 'date' or 'length' is null
missing_dates_or_lengths = img[img['date'].isnull() | img['length'].isnull()]
print(missing_dates_or_lengths)

                                                   body date  length
123   If summer is to bring any sense of normalcy th...  NaN    1511
500                                                 NaN  NaN       0
501                                                 NaN  NaN       0
1002                                                NaN  NaN       0
1003                                                NaN  NaN       0
1190  Stock and oil prices climbed on Monday on news...  NaN     653
1504                                                NaN  NaN       0
1505                                                NaN  NaN       0
2006                                                NaN  NaN       0
2007                                                NaN  NaN       0
2508                                                NaN  NaN       0
2509                                                NaN  NaN       0
3010                                                NaN  NaN       0
3011                              

In [66]:
img1 = img
from dateutil.parser import parse

def is_date(string):
    try: 
        parse(string)
        return True
    except ValueError:
        return False

# Convert all dates to strings before trying to parse them
img1['date'] = img1['date'].astype(str)

# Create a boolean mask for the rows with good dates
mask = img1['date'].apply(is_date)

# Keep only the rows with good dates (and drop the rows with bad dates)
img1 = img1[mask]
img1




,body,date,length
0,I note that Keir Starmer has opened his accoun...,6-Apr-20,270
1,As Covid19 makes social distancing a matter of...,6-Apr-20,963
2,Britons will need to stay at home for weeks to...,6-Apr-20,646
3,Labourâs new leader exudes competence and id...,6-Apr-20,613
4,WHEN are news and magazinestyle programmes goi...,6-Apr-20,102
...,...,...,...
3546,Thereâs no sign that ministers will use the ...,27-Dec-20,1065
3547,On this last Sunday of 2020 we reflect on what...,27-Dec-20,831
3548,sir I thought your Leading Article December 2...,27-Dec-20,165
3549,sir I have been following the discussion abou...,27-Dec-20,355


In [67]:
# Create a function to handle multiple date formats
def parse_dates(date):
    for fmt in ('%b%d,%Y', '%d-%b-%y'):
        try:
            return pd.datetime.strptime(date, fmt)
        except ValueError:
            continue
    return np.nan

# Apply the function to the date column
img1['date'] = img1['date'].apply(parse_dates)


img1.head()

C:\Users\20315\AppData\Local\Temp\ipykernel_17168\2742660590.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return pd.datetime.strptime(date, fmt)
C:\Users\20315\AppData\Local\Temp\ipykernel_17168\2742660590.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img1['date'] = img1['date'].apply(parse_dates)


,body,date,length
0,I note that Keir Starmer has opened his accoun...,2020-04-06,270
1,As Covid19 makes social distancing a matter of...,2020-04-06,963
2,Britons will need to stay at home for weeks to...,2020-04-06,646
3,Labourâs new leader exudes competence and id...,2020-04-06,613
4,WHEN are news and magazinestyle programmes goi...,2020-04-06,102


In [68]:
img1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3492 entries, 0 to 3550
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   body    3198 non-null   object        
 1   date    3488 non-null   datetime64[ns]
 2   length  3492 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 109.1+ KB


In [69]:
img1 = img1.dropna(subset=['date', 'length','body'])
img1.head()

,body,date,length
0,I note that Keir Starmer has opened his accoun...,2020-04-06,270
1,As Covid19 makes social distancing a matter of...,2020-04-06,963
2,Britons will need to stay at home for weeks to...,2020-04-06,646
3,Labourâs new leader exudes competence and id...,2020-04-06,613
4,WHEN are news and magazinestyle programmes goi...,2020-04-06,102


In [70]:
# Sort the DataFrame by date in ascending order
img1 = img1.sort_values(by='date')
img1 = img1.reset_index(drop=True)
# Check the result
img1.head()

,body,date,length
0,The novel coronavirus has killed at least 425 ...,2020-02-04,281
1,The novel coronavirus has killed at least 425 ...,2020-02-05,297
2,In just over a month the coronavirus outbreak ...,2020-02-18,470
3,The World Health Organization WHO still hasnt ...,2020-02-25,747
4,sir The Schengen Agreement is the perfect pol...,2020-02-25,376


In [71]:
img1['positive_word_count'] = img1['body'].apply(count_positive_words)
# Add a new column 'negative_word_count' to the DataFrame
img1['negative_word_count'] = img1['body'].apply(count_negative_words)

In [72]:
img1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194 entries, 0 to 3193
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   body                 3194 non-null   object        
 1   date                 3194 non-null   datetime64[ns]
 2   length               3194 non-null   int64         
 3   positive_word_count  3194 non-null   int64         
 4   negative_word_count  3194 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 124.9+ KB


In [73]:
columns_to_drop = ['body']
img1.drop(columns=columns_to_drop, inplace=True)
img1.head()

,date,length,positive_word_count,negative_word_count
0,2020-02-04,281,12,11
1,2020-02-05,297,13,11
2,2020-02-18,470,8,18
3,2020-02-25,747,8,11
4,2020-02-25,376,8,7


In [75]:
img_grouped = img1.groupby('date').agg({
    'length': 'sum',
    'positive_word_count': 'sum',
    'negative_word_count': 'sum'
}).reset_index()
img_grouped.to_csv('img_cleaned.csv', index=False)
img_grouped.head()

,date,length,positive_word_count,negative_word_count
0,2020-02-04,281,12,11
1,2020-02-05,297,13,11
2,2020-02-18,470,8,18
3,2020-02-25,1927,24,30
4,2020-02-28,2858,53,59


In [2]:
img_merged = img_grouped.merge(ny, on='date', how='inner')
img_merged['negative_score'] = img_merged['negative_word_count']/img_merged['length']
img_merged.tail()

NameError: name 'img_grouped' is not defined

In [78]:
##### import pandas as pd
import numpy as np
import statsmodels.api as sm

df = img_merged

# Assuming df is your DataFrame and 'Daily_return' and 'negative_word_count' are the columns with returns and sentiment scores respectively
df['return_lag1'] = df['Daily_return'].shift(1)
df['return_lag2'] = df['Daily_return'].shift(2)

# Drop the missing values that were created because of the lag
df = df.dropna()

# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     14.46
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           1.30e-06
Time:                        19:04:02   Log-Likelihood:                 520.45
No. Observations:                 216   AIC:                            -1035.
Df Residuals:                     213   BIC:                            -1025.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0008      0.001      0.544      

In [79]:
# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2', 'negative_score']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     9.627
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           5.50e-06
Time:                        19:04:08   Log-Likelihood:                 520.50
No. Observations:                 216   AIC:                            -1033.
Df Residuals:                     212   BIC:                            -1019.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0027      0.006      0.

# Full Corpus

In [111]:
full = pd.concat([clean_grouped, img_grouped])
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645 entries, 0 to 311
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 645 non-null    datetime64[ns]
 1   length               645 non-null    int64         
 2   positive_word_count  645 non-null    int64         
 3   negative_word_count  645 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 25.2 KB


In [112]:
full_grouped = full.groupby('date').agg({
    'length': 'sum',
    'positive_word_count': 'sum',
    'negative_word_count': 'sum'
}).reset_index()
full_grouped.to_csv('full_grouped.csv', index=False)
full_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 334 non-null    datetime64[ns]
 1   length               334 non-null    int64         
 2   positive_word_count  334 non-null    int64         
 3   negative_word_count  334 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 10.6 KB


In [114]:
full_merged = full_grouped.merge(ny, on='date', how='inner')
full_merged.head()

,date,length,positive_word_count,negative_word_count,Open,High,Low,Close,Adj Close,Volume,Daily_return,Trend,Trend_duration
0,2020-01-23,386,5,10,14064.280273,14109.589844,14003.280273,14102.040039,14102.040039,3766710000,-0.000581,DOWN,1
1,2020-01-27,1599,23,32,13746.629883,13826.429688,13742.009766,13769.599609,13769.599609,3831050000,-0.014942,DOWN,3
2,2020-01-28,1094,30,25,13812.650391,13913.589844,13798.339844,13877.610352,13877.610352,3531570000,0.007844,UP,1
3,2020-01-29,675,7,18,13912.790039,13922.440430,13843.790039,13843.809570,13843.809570,3600250000,-0.002436,DOWN,1
4,2020-01-31,530,0,14,13783.809570,13788.219727,13573.040039,13614.099609,13614.099609,4529700000,-0.017878,DOWN,1


In [115]:
full_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 0 to 233
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 234 non-null    datetime64[ns]
 1   length               234 non-null    int64         
 2   positive_word_count  234 non-null    int64         
 3   negative_word_count  234 non-null    int64         
 4   Open                 234 non-null    float64       
 5   High                 234 non-null    float64       
 6   Low                  234 non-null    float64       
 7   Close                234 non-null    float64       
 8   Adj Close            234 non-null    float64       
 9   Volume               234 non-null    int64         
 10  Daily_return         234 non-null    float64       
 11  Trend                234 non-null    object        
 12  Trend_duration       234 non-null    int64         
dtypes: datetime64[ns](1), float64(6), i

In [116]:
full_merged['negative_score'] = full_merged['negative_word_count']/full_merged['length']

In [117]:
##### import pandas as pd
import numpy as np
import statsmodels.api as sm

df = full_merged

# Assuming df is your DataFrame and 'Daily_return' and 'negative_word_count' are the columns with returns and sentiment scores respectively
df['return_lag1'] = df['Daily_return'].shift(1)
df['return_lag2'] = df['Daily_return'].shift(2)

# Drop the missing values that were created because of the lag
df = df.dropna()

# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     14.68
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           1.00e-06
Time:                        19:53:38   Log-Likelihood:                 563.10
No. Observations:                 232   AIC:                            -1120.
Df Residuals:                     229   BIC:                            -1110.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0004      0.001      0.306      

In [118]:
# Define your dependent variable (y) and independent variables (X)
y = df['Daily_return']
X = df[['return_lag1', 'return_lag2', 'negative_score']]

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Fit the model
model = sm.O
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Daily_return   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     10.41
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           1.90e-06
Time:                        19:53:49   Log-Likelihood:                 564.00
No. Observations:                 232   AIC:                            -1120.
Df Residuals:                     228   BIC:                            -1106.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0088      0.006      1.